# Audio - Super Pixels

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#|default_exp audio.superpixels
#|export
import cgnai
from pathlib import Path
import sys
from cgnai.logging import cgnai_logger
import numpy as np
from cgnai.fileio import ls, load

logger = cgnai_logger("superpixels")
log = logger.info

## Load Sample Data

In [ ]:
from cgnai.utils import cgnai_home
from cgnai.fileio import ls, load

data_path = cgnai_home() / "shared/podverse/data/dlf_politik_podcast/"
files = [f for f in ls(data_path).files if str(f).endswith(".mp3")]
len(files)

In [ ]:
from cgnai.audio.embeddings import load_embedding

emb = load_embedding(str(data_path / files[126]))
d=emb@emb.T

In [ ]:
from matplotlib import pyplot as plt

plt.hist(d.flatten(), bins=200);

In [ ]:
from scipy.stats import norm
from sklearn.mixture import GaussianMixture
from IPython.display import display, Math

gm = GaussianMixture(n_components=2, random_state=0).fit(d.reshape(-1, 1))

mean = gm.means_  
covs  = gm.covariances_
weights = gm.weights_

x_axis = np.arange(-0.25, 1, 0.01)
y_axis0 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
y_axis1 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[1][0][0])))*weights[1] # 2nd gaussian


plt.hist(d.flatten(), density=True, color='black', bins=100)
plt.plot(x_axis, y_axis0, lw=3, c='C0')
plt.plot(x_axis, y_axis1, lw=3, c='C1')
plt.plot(x_axis, y_axis0+y_axis1, lw=3, c='C2', ls='dashed')
plt.xlim(-0.25, 1)

display(Math(r'\mu_1: {} \\ \sigma_1: {} \\ \mu_2: {} \\ \sigma_2: {}'.format(mean[0][0], covs[0][0][0],mean[1][0], covs[1][0][0])))

# Super Pixels

In [ ]:
#|export
def var(i, j, d, var_cache=None):
    """
    calculates the mean variance between indices i and j
    """
    if (i,j) in var_cache:
        return var_cache[(i,j)]
    if i != j:
        x = d[i:j].var(axis=0)
        n = round(d.shape[1]*0.2)
        d_=np.mean(np.sort(x)[-n])
    else:
        d_ = 0.0
    var_cache[(i,j)] = d_
    return d_

def n_active(I_):
    """
    counts the number of active pixels in I
    """
    assert len(I_) > 1
    return sum([1 for i in range(0,len(I_)-1) if I_[i+1] != I_[i]])

def v_(I_, d, var_cache=None):
    return [var(I_[i],I_[i+1], d, var_cache) for i in range(0,len(I_)-1)]

def variance(I_, d, var_cache=None):
    return sum(v_(I_,d, var_cache))

def mean_variance(I_, d, var_cache=None):
    return sum(v_(I_,d, var_cache)) / n_active(I_)

def energy(I_, d, mu, var_cache=None):
    assert mu > 0
    return variance(I_, d, var_cache) + mu * n_active(I_)

In [ ]:
#|export
def find_super_pixels(d, N0=900, mu=0.0005, min_pixels_per_min=4, step=1, I0=None):
    var_cache = {}
    T = d.shape[0]
    I = [round(x) for x in np.linspace(0,T,N0)]
    min_pixels = round(T / 60 * min_pixels_per_min)
    log(f"min_pixels: {min_pixels}")
    n_updates = 1
    if I0 is not None:
        I = I0
    while n_updates > 0:
        n_updates = 0
        
        for i in range(1,len(I)-1):
            #i = np.random.randint(1,len(I)-1)
            # ---o-----o----o---
            #   x_L   x_1  x_R
            
            x_L = I[i-1]
            x_1 = I[i]
            x_R = I[i+1]

            e = energy([x_L, x_1, x_R], d, mu, var_cache)
            for x_1_ in range(x_L, x_R+1, step):
                    e_ = energy([x_L, x_1_, x_R], d, mu, var_cache)
                    if e_ < e:
                        e = e_
                        I[i] = x_1_
                        n_updates += 1
        
        to_delete = []
        for i in range(len(I)-2, -1, -1):
            if I[i] == I[i+1]:
                to_delete.append(i+1)
        for j in to_delete:
            del I[j]
        if len(I) < min_pixels:
            log("NOT CONVERGED")
            break
    return I

In [ ]:
#|export
def load_super_pixels(mp3_path):
    return load(str(mp3_path) + "_super_pixels.npy")

In [ ]:
#|export
from matplotlib import pyplot as plt
import matplotlib.patches as patches

def plot_super_pixels(I, d, x_min=None, x_max=None, mu=None, N0=None, N=None):
    fig, ax = plt.subplots(figsize=(10,10))
    plt.title(f'N:{n_active(I)}')
    if x_min is None:
        x_min = 0
    if x_max is None:
        x_max = d.shape[0]
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(x_max,x_min)
    ax.imshow(d, interpolation="none")
    for i in range(0,len(I)-1):
        ax.add_patch(patches.Rectangle((I[i], I[i]),I[i+1]-I[i], I[i+1]-I[i], linewidth=1, edgecolor='r', facecolor='none'))
    return fig

In [ ]:
N0 = round(20 * d.shape[0]/60)
I = find_super_pixels(d, N0=N0, mu=0.0001, min_pixels_per_min=2)
plot_super_pixels(I,d);

In [ ]:
plot_super_pixels(I,d);